<a href="https://colab.research.google.com/github/MattiaVerticchio/PersonalProjects/blob/master/NewsClassification/ReutersNewsClassification_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reuters News Classification
> [Italiano]() / **English**


> **Abstract**
>
>  The objective of this notebook is the classification of news by the Reuters agency. We’ll try a Neural Architecture Search approach for Deep Learning applied to Natural Language Processing to analyze strings of text and match them to the proper news category. The benchmark used as a reference is the textbook [Deep Learning with Python by Francois Chollet](https://www.manning.com/books/deep-learning-with-python). The results have accuracy in line with the benchmark, and the target metric, categorical cross-entropy is outperformed by almost 3% within a few minutes of training.

## Introduction
As the central framework for this task, we’ll use [Auto-Keras](https://arxiv.org/abs/1806.10282), an efficient neural architecture search system developed by DATA Lab at Texas A&M University. It leverages a variant of Bayesian Optimization to guide deep neural network morphism and find a good architecture for our task and dataset, using [Keras](https://keras.io/) and [TensorFlow](https://www.tensorflow.org/) as backend.
### Framework Setup
First, we have to install it with its dependency, [Keras-Tuner](https://keras-team.github.io/keras-tuner/), which is the hyperparameter optimization library used by [Auto-Keras](https://arxiv.org/abs/1806.10282).

In [ ]:
%%bash
pip install -q git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1
pip install -q autokeras

### Imports
Now we can import the TensorFlow dataset loading tool and Auto-Keras, as well as NumPy to store images and labels in arrays.

In [ ]:
from tensorflow.keras.datasets import reuters
import autokeras as ak
import numpy as np

### Loading the data
The dataset can be now loaded into four NumPy arrays.

The data has an offset of 3 units, the first three indices are mapped as follows.
```
0. Padding
1. Start of sequence
2. Unknown word
```

In [ ]:
offset = 3
(x_train, y_train), (x_test, y_test) =  reuters.load_data(num_words=1000,
                                                          index_from=offset)
y_train = y_train.reshape(-1, 1)
y_test  = y_test.reshape(-1, 1)

2113536/2110848 [==============================] - 0s 0us/step


Let’s prepare the dictionary that maps indices to words.

In [ ]:
word_to_id = reuters.get_word_index()
word_to_id = {k: (v + offset) for k, v in word_to_id.items()}
word_to_id['PADDING'] = 0
word_to_id['START_OF_SEQUENCE'] = 1
word_to_id['UNKNKOWN'] = 2
id_to_word = {value: key for key, value in word_to_id.items()}

557056/550378 [==============================] - 0s 0us/step


Now we convert the lists of indices to actual words using a dictionary map. 

In [ ]:
x_train = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_train))
x_test = list(map(lambda sentence: ' '.join(
    id_to_word[i] for i in sentence), x_test))

x_train = np.array(x_train, dtype=np.str)
x_test  = np.array(x_test, dtype=np.str)

## Building the model
`TextClassifier` is the class responsible for the model search. Here I set `max_trials=3` to avoid taking too much time for model exploration, but it can be set to any positive integer. It explores different model architectures by tree-based Bayesian Optimization search. We’ll try only the first three as it’s time-consuming on [Google Colab](https://colab.research.google.com/)’s GPU, where I’m running this notebook.

In [ ]:
clf = ak.TextClassifier(              # Initialize the text classifier
    num_classes=None,                 # Infer the number of classes
    multi_label=False,                # Only one output
    loss='categorical_crossentropy',  # Select the loss metric
    metrics='accuracy',               # Metric to watch
    project_name="text_classifier",   # Name of the folder
    max_trials=3,                     # Just try three models
    directory=None,                   # Automatic folder creation
    objective="val_loss",             # Validation set crossentropy
    tuner=None,                       # Automatic hyperparameter tuner selection
    overwrite=True,                   # Don't load previous experiments
    seed=42                           # Set a seed to replicate the experiment
)

### Neural Architecture Search
It’s all ready to start exploring the models.

In [ ]:
clf.fit(                   # Fit the model
    x=x_train,             # Training features
    y=y_train,             # Training labels
    epochs=None,           # Automatic number of epochs
    callbacks=None,        # No callbacks
    validation_split=0.2,  # Validation data split
    validation_data=None   # Use a portion of training data
)

Trial 3 Complete [00h 01m 27s]
val_loss: 1.2090760469436646

Best val_loss So Far: 0.9410121440887451
Total elapsed time: 00h 04m 35s
INFO:tensorflow:Oracle triggered exit
Epoch 1/9
281/281 [==============================] - 4s 13ms/step - loss: 2.0264 - accuracy: 0.4977
Epoch 2/9
281/281 [==============================] - 3s 12ms/step - loss: 1.3888 - accuracy: 0.6778
Epoch 3/9
281/281 [==============================] - 3s 12ms/step - loss: 1.1446 - accuracy: 0.7269
Epoch 4/9
281/281 [==============================] - 3s 12ms/step - loss: 0.9995 - accuracy: 0.7621
Epoch 5/9
281/281 [==============================] - 4s 13ms/step - loss: 0.8872 - accuracy: 0.7867
Epoch 6/9
281/281 [==============================] - 3s 12ms/step - loss: 0.7960 - accuracy: 0.8042
Epoch 7/9
281/281 [==============================] - 3s 12ms/step - loss: 0.7154 - accuracy: 0.8219
Epoch 8/9
281/281 [==============================] - 3s 12ms/step - loss: 0.6371 - accuracy: 0.8390
Epoch 9/9
281/281 [=========

### Model architecture
The model found can now be exported. It’s a convolutional multi-layer neural network, and it has the following architecture.

In [ ]:
model = clf.export_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization (TextVect (None, 512)               0         
_________________________________________________________________
embedding (Embedding)        (None, 512, 64)           320064    
_________________________________________________________________
dropout (Dropout)            (None, 512, 64)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 508, 256)          82176     
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)              

## Testing the model
The benchmark chosen for this dataset is the experiment reported in the book [Deep Learning with Python by Francois Chollet](https://www.manning.com/books/deep-learning-with-python).

The testing categorical cross-entropy of the benchmark is ~0.96.

Let’s now see how our model performs.

In [ ]:
current  = clf.evaluate(x_test, y_test)
previous = 0.9565213431445807

improvement = (abs(current[0] - previous) / previous) * 100
print(f'The categorical crossentropy improvement is {round(improvement, 1)}%.')

71/71 [==============================] - 0s 6ms/step - loss: 0.9285 - accuracy: 0.7850
The categorical crossentropy improvement is 2.9%.


The accuracy of ~0.79 is in line with the previous model. However, we scored a cross-entropy of ~0.93 with only three models explored, outperforming the textbook benchmark by almost 3%.

Depending on our hardware and time availability, of course, we could explore even more models for further improvement in the benchmark score.

### [**Go back to index >**](https://github.com/MattiaVerticchio/PersonalProjects/blob/master/README_EN.md)